In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import requests
from talib import abstract
import math

plt.style.use('fivethirtyeight')

In [2]:
# data_request = requests.get('https://api3.binance.com/api/v3/klines?symbol=ETHBUSD&interval=5m&limit=1000')
# bnb_btc_data = data_request.json()
# data = pd.DataFrame(data = {
#     'open time': [],
#     'open': [],
#     'high': [],
#     'low': [],
#     'close': [],
# })

# for kline_item in bnb_btc_data :
#     time = kline_item[0] / 1000
#     row = {
#         'open time': datetime.utcfromtimestamp(time),
#         'open': float(kline_item[1]),
#         'high': float(kline_item[2]),
#         'low': float(kline_item[3]),
#         'close': float(kline_item[4]),
#     }
#     data = data.append(row, ignore_index=True)


# data.head()

# data.to_csv("working-data.csv", index=False)

data = pd.read_csv('ETHBUSD.csv')
data = data.iloc[-4880:-1]
data = data.reset_index()

In [3]:
data['sma144'] = abstract.SMA(data, timeperiod=144)
data['sma3'] = abstract.SMA(data, timeperiod=3)
data['sma20'] = abstract.SMA(data, timeperiod=20)

macd = abstract.MACD(data, fastperiod=12, slowperiod=26, signalperiod=9)
data['macd'] = macd['macd']
data['macdsignal'] = macd['macdsignal']
data['macdhist'] = macd['macdhist']
data['rsi'] = abstract.RSI(data, timeperiod=24)
data.to_csv("data.csv", index=False)

In [8]:

money = 10000
    

earns = pd.DataFrame(data = {
    'buy time': [],
    'buy': [],
    'sell': [],
    'sell time': [],
    'shares': [],
    'earn': [],
})

flag = 'sold'

for i in range(len(data)):
    if i <= 144:
        continue
        
    current_value = data.iloc[i]
        
    if flag == 'sold':
        is_up_trend = current_value['close'] > current_value['sma144']
        has_crossed = current_value['sma3'] > current_value['sma20']
        # is_current_up = current_value['close'] > current_value['sma3']
        
        buy_signal = is_up_trend and has_crossed
        
        if buy_signal:
            flag = 'bought'
            earns = earns.append({ 
                'buy': current_value['close'],
                'buy time': current_value['open time'],
                'shares': money / current_value['close']
            }, ignore_index=True)
            continue

    elif flag == 'bought':
        
        rowIndex = earns[earns['buy'].notna() & earns['sell'].isna()].index.item()
        money_diff = (current_value['close'] / earns['buy'][rowIndex]) - 1
        
        if money_diff >= 0.0075 or money_diff <= -0.005:
            rowIndex = earns[earns['buy'].notna() & earns['sell'].isna()].index.item()
            flag = 'sold'
            
            lucro = earns.loc[rowIndex, 'shares'] * current_value['close']
            
            earns.loc[rowIndex, 'sell'] = current_value['close']
            earns.loc[rowIndex, 'earn'] = lucro - money 
            earns.loc[rowIndex, 'sell time'] = current_value['open time']
            continue
    else:
        continue
   
         
print('Lucro', earns['earn'].sum())
total = earns['earn'].count()
print('Quantidade', earns['earn'].count())
wins = earns[earns['earn'] > 0].count()
p = (wins/total)['earn'] * 100
print(f'Porcentagem {p}%')
print('-------')
earns.to_csv("result.csv", index=False)

Lucro -89.16545958117058
Quantidade 33
Porcentagem 48.484848484848484%
-------
